In [ ]:
import warnings
from typing import Any
import joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, average_precision_score

from utils import plot_roc
import ml_utils as utils
from ml_utils import get_weights_for_roc_auc, weighted_roc_metric, get_optimal_trs, plot_roc
import train


np.random.seed(42)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# aft, magn > 6

## read 

In [2]:
dataset_type_train = "magn_6_aft"
dataset_type_test = "magn_6_aft_test"

target = "target"
dt_col = "dt"
min_test_date = "2020-10-14"
min_train_date = "2014-03-17"

train_path = f"data/dataset/{dataset_type_train}_trs6.0.parquet"
test_path = f"data/dataset/{dataset_type_test}_trs6.0.parquet"

In [ ]:
train_df, test_df = utils.get_train_test_datasets(
    train_path=train_path, test_path=test_path, 
    min_test_date = min_test_date, min_train_date = min_train_date
)

In [4]:
feature_cols = list(set(train_df.columns) - set([target, dt_col, "horizon_min_dt", "horizon_max_dt"]))

In [5]:
# train, test
features = utils.Features(
    features=feature_cols, target=target, dt_col=dt_col, 
    groupby=["cell_x", "cell_y"], target_features=[]
)

X_train, y_train = features.make_features_target(train_df)
X_test, y_test = features.make_features_target(test_df)

In [ ]:
y_test.reset_index().merge(y_train.reset_index(), how="left").isna().sum()

In [7]:
# get weights for roc auc as probability density of occured earthquakes
roc_auc_weights_test = get_weights_for_roc_auc(y=y_test, last_dt=min_test_date)
roc_auc_weights_train = get_weights_for_roc_auc(y=y_train, last_dt=min_test_date)

In [8]:
assert len(roc_auc_weights_test) == len(roc_auc_weights_test[~np.isnan(roc_auc_weights_test)]) == len(y_test)
assert len(roc_auc_weights_train) == len(roc_auc_weights_train[~np.isnan(roc_auc_weights_train)]) == len(y_train)

In [ ]:
roc_auc_weights_test

In [ ]:
len(roc_auc_weights_test[roc_auc_weights_test > 0.0]), np.max(roc_auc_weights_test), np.unique(roc_auc_weights_test)

In [ ]:
plt.hist(roc_auc_weights_test[roc_auc_weights_test > 0.0], range=(0, 1.0), bins=20);

In [12]:
y_train = y_train[target]
y_test = y_test[target]

In [ ]:
y_train.sum(), y_test.sum()

## lgbm

In [ ]:
cv = utils.MonthlyTimeSeriesSplit(window=20)
cv.get_n_splits(X_train)

In [ ]:
for split_train, split_test in cv.split(X_train):
    train_start = np.sort(X_train.iloc[split_train].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[0]
    train_end = np.sort(X_train.iloc[split_train].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[-1]
    test_start = np.sort(X_train.iloc[split_test].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[0]
    test_end = np.sort(X_train.iloc[split_test].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[-1]

    print(f"train: [{train_start}, {train_end}], len: {len(split_train)} | test: [{test_start}, {test_end}], len: {len(split_test)}")

### baseline

In [ ]:
params = {
    "n_estimators": 5,
    "objective": "binary",
    "use_missing": False,
    "deterministic": True,
    "random_state": 42,
    "force_col_wise": True,
    "feature_pre_filter": False,
    "verbosity": -1,
    "max_depth": 2,
    "n_jobs": 50,
}

lgbm_model = train.get_lgbm_model(feature_cols, params)

lgbm_model = lgbm_model.fit(X_train, y_train)
y_pred_prob = lgbm_model.predict_proba(X_test)

baseline_roc_auc = roc_auc_score(y_test, y_pred_prob[:, 1])

(
    roc_auc_score(y_train, lgbm_model.predict_proba(X_train)[:, 1]), 
    roc_auc_score(y_test, y_pred_prob[:, 1]),
    "----",
    weighted_roc_metric(y_train, lgbm_model.predict_proba(X_train)[:, 1]),
    weighted_roc_metric(y_test, lgbm_model.predict_proba(X_test)[:, 1])
)

### shap feature selection

In [ ]:
from sklearn.metrics import make_scorer
from probatus.utils import Scorer


weighted_roc_auc_sklearn_scorer = make_scorer(weighted_roc_metric, greater_is_better=True)
weighted_roc_auc_probatus_scorer = Scorer("weighted_roc_auc", custom_scorer=weighted_roc_auc_sklearn_scorer)

shap_params = params.copy()
shap_params["n_jobs"] = 5

lgbm_reg = lgb.LGBMRegressor(**shap_params)

imputer = SimpleImputer(
    strategy="constant",
    fill_value=0.0,
).set_output(transform="pandas")
X_train_preprocessed = imputer.fit_transform(X_train)

shap_elimination, lgbm_shap_features = train.select_features(
    model=lgbm_reg,
    X=X_train_preprocessed,
    y=y_train,
    cv=cv,
    n_jobs=20,
    metric=weighted_roc_auc_probatus_scorer,
    step=0.05,
    return_rfe=True,
)

In [ ]:
lgbm_shap_features

In [ ]:
print(len(lgbm_shap_features))
utils.get_features_dict(lgbm_shap_features)

In [ ]:
lgbm_model = train.get_lgbm_model(lgbm_shap_features, shap_params)

lgbm_model = lgbm_model.fit(X_train, y_train)
y_pred_prob = lgbm_model.predict_proba(X_test)

(
    roc_auc_score(y_train, lgbm_model.predict_proba(X_train)[:, 1]), 
    roc_auc_score(y_test, y_pred_prob[:, 1]),
    "----",
    weighted_roc_metric(y_train, lgbm_model.predict_proba(X_train)[:, 1]),
    weighted_roc_metric(y_test, lgbm_model.predict_proba(X_test)[:, 1])
)

### hyper optimiziation

In [ ]:
import optuna
import optuna.trial
import optuna.logging

optuna.logging.set_verbosity(optuna.logging.WARNING)


def fixed_params(**kwargs) -> dict[str, Any]:
    params = {
        "objective": "binary",
        "use_missing": False,
        "deterministic": True,
        "random_state": 42,
        "force_col_wise": True,
        "feature_pre_filter": False,
        "verbosity": -1,
        "n_jobs": 1,
    }

    params.update(kwargs)

    return params


def default_params(**kwargs) -> dict[str, Any]:
    params = {
        **fixed_params(),
        "colsample_bytree": 1.0,
        "subsample": 1.0,
        "learning_rate": 0.01,
        "num_leaves": 31,
        "min_child_samples": 20,
        "n_estimators": 10,
        "max_depth": 2,
    }
    params.update(kwargs)

    return params


def suggest_params(trial: optuna.trial.Trial, **kwargs) -> dict[str, Any]:
    params = {
        **fixed_params(),
        "n_estimators": trial.suggest_int("n_estimators", 3, 10, step=1),
        "max_depth": trial.suggest_int("max_depth", 1, 2, step=1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, step=0.001),
        "num_leaves": trial.suggest_int("num_leaves", 5, 50),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 29.6, step=0.5),
    }
    params.update(kwargs)

    return params

tuned_lgbm = train.run_optuna(
    X = X_train,
    y = y_train,
    n_trials = 500,
    dump_study_path = f"data/optuna_res/lgbm_{dataset_type_train.replace('_train', '')}",
    seed = 42,
    resume = False,
    suggest_params = suggest_params,
    default_params = default_params,
    cv = utils.MonthlyTimeSeriesSplit(window=20, partition=True),
    model = train.get_lgbm_model(lgbm_shap_features, default_params()),
    n_jobs = 50,
    model_name = "lgbmclassifier",
)

In [ ]:
y_train

In [ ]:
tuned_lgbm_params = tuned_lgbm["lgbmclassifier"].get_params()
tuned_lgbm_params

In [ ]:
reulst_aft_lgbm = tuned_lgbm.fit(X_train, y_train)
y_pred_prob = reulst_aft_lgbm.predict_proba(X_test)

(
    roc_auc_score(y_train, reulst_aft_lgbm.predict_proba(X_train)[:, 1]), 
    roc_auc_score(y_test, y_pred_prob[:, 1]),
    "----",
    weighted_roc_metric(y_train, reulst_aft_lgbm.predict_proba(X_train)[:, 1]),
    weighted_roc_metric(y_test, y_pred_prob[:, 1])
)

In [ ]:
plot_roc(target=y_test, prediction=y_pred_prob[:, 1], weights=None);

In [ ]:
plot_roc(target=y_test, prediction=y_pred_prob[:, 1], weights=roc_auc_weights_test, src_label="weighted");

In [ ]:
t = get_optimal_trs(y_test.astype(int), y_pred_prob[:, 1], None)
print("optimal trs:", t)

binary_preds = y_pred_prob[:, 1].copy()
binary_preds[y_pred_prob[:, 1] >= t] = 1
binary_preds[y_pred_prob[:, 1] < t] = 0
precision_recall_fscore_support(y_test.astype(int), binary_preds.astype(int))

In [ ]:
t = get_optimal_trs(y_test.astype(int), y_pred_prob[:, 1], sample_weight=roc_auc_weights_test)
print("optimal trs:", t)

binary_preds = y_pred_prob[:, 1].copy()
binary_preds[y_pred_prob[:, 1] >= t] = 1
binary_preds[y_pred_prob[:, 1] < t]
precision_recall_fscore_support(y_test.astype(int), binary_preds.astype(int), sample_weight=roc_auc_weights_test)

In [29]:
with open("data/models/aft_6_lgbm.pickle", "xb") as f:
    joblib.dump(reulst_aft_lgbm, f)

# withoutaft, magn > 6

## read 

In [ ]:
dataset_type_train = "magn_6_withoutaft"
dataset_type_test = "magn_6_withoutaft_test"

target = "target"
dt_col = "dt"
min_test_date = "2020-10-14"
min_train_date = "2014-03-17"

train_path = f"data/dataset/{dataset_type_train}_trs6.0.parquet"
test_path = f"data/dataset/{dataset_type_test}_trs6.0.parquet"

In [ ]:
train_df, test_df = utils.get_train_test_datasets(
    train_path=train_path, test_path=test_path, 
    min_test_date = min_test_date, min_train_date = min_train_date
)

In [ ]:
feature_cols = list(set(train_df.columns) - set([target, dt_col, "horizon_min_dt", "horizon_max_dt"]))

In [ ]:
# train, test
features = utils.Features(
    features=feature_cols, target=target, dt_col=dt_col, 
    groupby=["cell_x", "cell_y"], target_features=[]
)

X_train, y_train = features.make_features_target(train_df)
X_test, y_test = features.make_features_target(test_df)

In [ ]:
y_test.reset_index().merge(y_train.reset_index(), how="left").isna().sum()

In [ ]:
# get weights for roc auc as probability density of occured earthquakes
roc_auc_weights_test = get_weights_for_roc_auc(y=y_test, last_dt=min_test_date)
roc_auc_weights_train = get_weights_for_roc_auc(y=y_train, last_dt=min_test_date)

In [ ]:
assert len(roc_auc_weights_test) == len(roc_auc_weights_test[~np.isnan(roc_auc_weights_test)]) == len(y_test)
assert len(roc_auc_weights_train) == len(roc_auc_weights_train[~np.isnan(roc_auc_weights_train)]) == len(y_train)

In [ ]:
len(roc_auc_weights_test[roc_auc_weights_test > 0.0]), np.max(roc_auc_weights_test), np.unique(roc_auc_weights_test)

In [ ]:
plt.hist(roc_auc_weights_test[roc_auc_weights_test > 0.0], range=(0, 1.0), bins=20);

In [ ]:
y_train = y_train[target]
y_test = y_test[target]

In [ ]:
y_train.sum(), y_test.sum()

## lgbm

In [ ]:
cv = utils.MonthlyTimeSeriesSplit(window=20)
cv.get_n_splits(X_train)

In [ ]:
for split_train, split_test in cv.split(X_train):
    train_start = np.sort(X_train.iloc[split_train].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[0]
    train_end = np.sort(X_train.iloc[split_train].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[-1]
    test_start = np.sort(X_train.iloc[split_test].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[0]
    test_end = np.sort(X_train.iloc[split_test].drop(columns=["cell_x", "cell_y"]).reset_index()["dt"].unique())[-1]

    print(f"train: [{train_start}, {train_end}], len: {len(split_train)} | test: [{test_start}, {test_end}], len: {len(split_test)}")

### baseline

In [ ]:
params = {
    "n_estimators": 5,
    "objective": "binary",
    "use_missing": False,
    "deterministic": True,
    "random_state": 42,
    "force_col_wise": True,
    "feature_pre_filter": False,
    "verbosity": -1,
    "max_depth": 2,
    "n_jobs": 50,
}

lgbm_model = train.get_lgbm_model(feature_cols, params)

lgbm_model = lgbm_model.fit(X_train, y_train)
y_pred_prob = lgbm_model.predict_proba(X_test)

baseline_roc_auc = roc_auc_score(y_test, y_pred_prob[:, 1])

(
    roc_auc_score(y_train, lgbm_model.predict_proba(X_train)[:, 1]), 
    roc_auc_score(y_test, y_pred_prob[:, 1]),
    "----",
    weighted_roc_metric(y_train, lgbm_model.predict_proba(X_train)[:, 1]),
    weighted_roc_metric(y_test, lgbm_model.predict_proba(X_test)[:, 1])
)

### shap feature selection

In [ ]:
shap_params = params.copy()
shap_params["n_jobs"] = 5

# cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
lgbm_reg = lgb.LGBMRegressor(**shap_params)

imputer = SimpleImputer(
    strategy="constant",
    fill_value=0.0,
).set_output(transform="pandas")
X_train_preprocessed = imputer.fit_transform(X_train)

shap_elimination, lgbm_shap_features = train.select_features(
    model=lgbm_reg,
    X=X_train_preprocessed,
    y=y_train,
    cv=cv,
    n_jobs=20,
    metric=weighted_roc_auc_probatus_scorer,
    step=0.05,
    return_rfe=True,
)

In [ ]:
lgbm_shap_features

In [ ]:
print(len(lgbm_shap_features))
utils.get_features_dict(lgbm_shap_features)

In [ ]:
lgbm_model = train.get_lgbm_model(lgbm_shap_features, shap_params)

lgbm_model = lgbm_model.fit(X_train, y_train)
y_pred_prob = lgbm_model.predict_proba(X_test)

(
    roc_auc_score(y_train, lgbm_model.predict_proba(X_train)[:, 1]), 
    roc_auc_score(y_test, y_pred_prob[:, 1]),
    "----",
    weighted_roc_metric(y_train, lgbm_model.predict_proba(X_train)[:, 1]),
    weighted_roc_metric(y_test, lgbm_model.predict_proba(X_test)[:, 1])
)

### hyper optimiziation

In [ ]:
import optuna
import optuna.trial
import optuna.logging

optuna.logging.set_verbosity(optuna.logging.WARNING)


def fixed_params(**kwargs) -> dict[str, Any]:
    params = {
        "objective": "binary",
        "use_missing": False,
        "deterministic": True,
        "random_state": 42,
        "force_col_wise": True,
        "feature_pre_filter": False,
        "verbosity": -1,
        "n_jobs": 1,
    }

    params.update(kwargs)

    return params


def default_params(**kwargs) -> dict[str, Any]:
    params = {
        **fixed_params(),
        "colsample_bytree": 1.0,
        "subsample": 1.0,
        "learning_rate": 0.01,
        "num_leaves": 31,
        "min_child_samples": 20,
        "n_estimators": 10,
        "max_depth": 2,
    }
    params.update(kwargs)

    return params


def suggest_params(trial: optuna.trial.Trial, **kwargs) -> dict[str, Any]:
    params = {
        **fixed_params(),
        "n_estimators": trial.suggest_int("n_estimators", 3, 10, step=1),
        "max_depth": trial.suggest_int("max_depth", 1, 2, step=1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, step=0.001),
        "num_leaves": trial.suggest_int("num_leaves", 5, 50),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 29.6, step=0.5),
    }
    params.update(kwargs)

    return params

tuned_lgbm = train.run_optuna(
    X = X_train,
    y = y_train,
    n_trials = 500,
    dump_study_path = f"data/optuna_res/lgbm_{dataset_type_train.replace('_train', '')}",
    seed = 42,
    resume = False,
    suggest_params = suggest_params,
    default_params = default_params,
    cv = utils.MonthlyTimeSeriesSplit(window=20, partition=True),
    model = train.get_lgbm_model(lgbm_shap_features, default_params()),
    n_jobs = 50,
    model_name = "lgbmclassifier",
)

In [ ]:
tuned_lgbm_params = tuned_lgbm["lgbmclassifier"].get_params()
tuned_lgbm_params

In [ ]:
reulst_witoutaft_lgbm = tuned_lgbm.fit(X_train, y_train)
y_pred_prob = reulst_witoutaft_lgbm.predict_proba(X_test)

(
    roc_auc_score(y_train, reulst_witoutaft_lgbm.predict_proba(X_train)[:, 1]), 
    roc_auc_score(y_test, y_pred_prob[:, 1]),
    "----",
    weighted_roc_metric(y_train, reulst_witoutaft_lgbm.predict_proba(X_train)[:, 1]),
    weighted_roc_metric(y_test, reulst_witoutaft_lgbm.predict_proba(X_test)[:, 1])
)

In [ ]:
plot_roc(target=y_test, prediction=y_pred_prob[:, 1], weights=None);

In [ ]:
plot_roc(target=y_test, prediction=y_pred_prob[:, 1], weights=roc_auc_weights_test, src_label="weighted");

In [ ]:
t = get_optimal_trs(y_test.astype(int), y_pred_prob[:, 1], None)
print("optimal trs:", t)

binary_preds = y_pred_prob[:, 1].copy()
binary_preds[y_pred_prob[:, 1] >= t] = 1
binary_preds[y_pred_prob[:, 1] < t]
precision_recall_fscore_support(y_test.astype(int), binary_preds.astype(int))

In [ ]:
t = get_optimal_trs(y_test.astype(int), y_pred_prob[:, 1], sample_weight=roc_auc_weights_test)
print("optimal trs:", t)

binary_preds = y_pred_prob[:, 1].copy()
binary_preds[y_pred_prob[:, 1] >= t] = 1
binary_preds[y_pred_prob[:, 1] < t]
precision_recall_fscore_support(y_test.astype(int), binary_preds.astype(int), sample_weight=roc_auc_weights_test)

In [55]:
with open("data/models/witoutaft_6_lgbm.pickle", "xb") as f:
    joblib.dump(reulst_witoutaft_lgbm, f)